In [ ]:
# Plant Disease Classification - Data Pipeline

This notebook demonstrates the use of refactored data modules for the PlantVillage dataset.

**Modules:**
- `data.dataset`: Custom PyTorch Dataset and DataLoader creation
- `data.transforms`: Image augmentation and normalization
- `data.utils`: Dataset splitting and subset creation utilities

In [ ]:
# Standard imports
import os
import sys
import yaml

# Add parent directory to path to import data modules
sys.path.insert(0, os.path.abspath('..'))

# Import our custom data modules
from data.dataset import MultiModalityDataset, create_dataloaders
from data.transforms import get_transforms
from data.utils import build_class_mapping, gather_samples, split_dataset, make_subset

import torch
from torch.utils.data import DataLoader

# ===========================
# Load Configuration from YAML
# ===========================
CONFIG_PATH = "../configs/train.yaml"

with open(CONFIG_PATH, 'r') as f:
    config = yaml.safe_load(f)

# Extract config values
DATA_DIR = config['data']['data_dir']
MODALITIES = config['data']['modalities']
IMAGE_SIZE = config['data']['image_size']
BATCH_SIZE = config['train']['batch_size']
NORMALIZE = config['data']['normalize']
AUGMENT = config['data']['augment']

print("✅ Configuration loaded successfully!")
print(f"   Data directory: {DATA_DIR}")
print(f"   Image size: {IMAGE_SIZE}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Normalize: {NORMALIZE}")
print(f"   Augment: {AUGMENT}")

In [ ]:
# 1️⃣ Build class name → ID mapping
class_names, class_to_idx = build_class_mapping(DATA_DIR, modality="color")

# 2️⃣ Gather all samples (paths + labels + modality)
samples = gather_samples(DATA_DIR, MODALITIES, class_to_idx)

print(f"Total samples found: {len(samples)}")
print(f"Number of classes: {len(class_names)}")
print(f"Example classes: {class_names[:5]}")

In [ ]:
# 3️⃣ Train/Val/Test split (using config values)
train, val, test = split_dataset(
    samples, 
    test_size=config['data']['test_size'], 
    val_size=config['data']['val_size']
)

print(f"Train: {len(train)}, Val: {len(val)}, Test: {len(test)}")

In [ ]:
# 4️⃣ Build datasets (using config for transforms)
train_dataset = MultiModalityDataset(
    train, 
    get_transforms(IMAGE_SIZE, train=True, normalize=NORMALIZE, augment=AUGMENT)
)
val_dataset = MultiModalityDataset(
    val, 
    get_transforms(IMAGE_SIZE, train=False, normalize=NORMALIZE, augment=False)
)
test_dataset = MultiModalityDataset(
    test, 
    get_transforms(IMAGE_SIZE, train=False, normalize=NORMALIZE, augment=False)
)


# 5️⃣ DataLoaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=config['data']['num_workers']
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=config['data']['num_workers']
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=config['data']['num_workers']
)


print("✅ DataLoaders are ready!")

In [ ]:
# 6️⃣ Optional: Create subsets for quick testing/prototyping

# Use subset_fraction from config if specified
if config['data'].get('subset_fraction') and config['data']['subset_fraction'] < 1.0:
    train = make_subset(train, config['data']['subset_fraction'])
    print(f"Using {config['data']['subset_fraction']*100:.0f}% subset: {len(train)} samples")

# Small subset for quick testing
train_tiny = make_subset(train, 0.05)
train_tiny_dataset = MultiModalityDataset(
    train_tiny, 
    get_transforms(IMAGE_SIZE, train=True, normalize=NORMALIZE, augment=AUGMENT)
)
train_tiny_loader = DataLoader(
    train_tiny_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=config['data']['num_workers']
)

# Medium subset for hyperparameter tuning
train_medium = make_subset(train, 0.3)
train_medium_dataset = MultiModalityDataset(
    train_medium, 
    get_transforms(IMAGE_SIZE, train=True, normalize=NORMALIZE, augment=AUGMENT)
)
train_medium_loader = DataLoader(
    train_medium_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=config['data']['num_workers']
)

print(f"Tiny subset: {len(train_tiny)} samples")
print(f"Medium subset: {len(train_medium)} samples")

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import vit_b_16, ViT_B_16_Weights
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import copy
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# 1️⃣ Load pretrained ViT
weights = ViT_B_16_Weights.IMAGENET1K_V1
model = vit_b_16(weights=weights)


# 2️⃣ Freeze feature layers (freeze everything except head)
for param in model.parameters():
    param.requires_grad = False


# 3️⃣ Replace the classification head
num_classes = len(class_names)  # from previous cell
model.heads = nn.Sequential(
    nn.Linear(model.heads.head.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, num_classes)
)

model.to(device)


# 4️⃣ Define Loss & Optimizer (only head parameters train)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.heads.parameters(), lr=1e-4)
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)


# 5️⃣ Training + Validation Loop
SAVE_PATH = "best_vit_model.pth"  # Saved in working directory


def train_model(num_epochs=1000, patience=100):
    best_val_loss = float("inf")
    best_model_wts = copy.deepcopy(model.state_dict())
    no_improve_epochs = 0

    for epoch in range(num_epochs):
        # ---------- Training ----------
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for batch in train_loader:
            images = batch["image"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            train_correct += predicted.eq(labels).sum().item()
            train_total += labels.size(0)

        train_loss /= train_total
        train_acc = train_correct / train_total

        # ---------- Validation ----------
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch in val_loader:
                images = batch["image"].to(device)
                labels = batch["label"].to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = val_correct / val_total

        # ---------- Scheduler step ----------
        scheduler.step()

        # ---------- Best model save ----------
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, SAVE_PATH)
            no_improve_epochs = 0
            improved = "✅ (improved & saved)"
        else:
            no_improve_epochs += 1
            improved = ""

        print(f"Epoch [{epoch+1}/{num_epochs}] "
              f"| Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
              f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} "
              + improved)

        # ---------- Optional Early Stopping ----------
        if patience is not None and no_improve_epochs >= patience:
            print(f"⏹ Early stopping at epoch {epoch+1} — no improvement for {patience} epochs.")
            break

    print("🏁 Training finished!")

    # Load best weights before returning
    model.load_state_dict(best_model_wts)
    return model


model = train_model(num_epochs=1000, patience=100)
print("✅ Best model restored & ready!")


## 🔍 Class Distribution Analysis

Let's check if our dataset has class imbalance issues.

In [ ]:
from data.utils import get_class_distribution, balance_dataset_uniform, calculate_class_weights, get_sample_weights

# Check class distribution
train_dist = get_class_distribution(train)
print(f"Class distribution in training set:")
print(f"  Min samples: {min(train_dist.values())}")
print(f"  Max samples: {max(train_dist.values())}")
print(f"  Mean samples: {sum(train_dist.values()) / len(train_dist):.1f}")
print(f"  Imbalance ratio: {max(train_dist.values()) / min(train_dist.values()):.2f}x")

## ⚖️ Strategy 1: Uniform Sampling (Downsample to Smallest Class)

This approach ensures equal representation by randomly downsampling larger classes.

In [ ]:
# Balance by uniform sampling
train_balanced = balance_dataset_uniform(train, seed=42)

# Verify balance
balanced_dist = get_class_distribution(train_balanced)
print(f"Balanced dataset size: {len(train_balanced)} (was {len(train)})")
print(f"Samples per class: {list(balanced_dist.values())[:5]}...")  # Show first 5
print(f"All classes equal: {len(set(balanced_dist.values())) == 1}")

## ⚖️ Strategy 2: Weighted Random Sampler (Oversample Small Classes)

This approach uses PyTorch's WeightedRandomSampler to give rare classes higher sampling probability.

In [ ]:
# Calculate per-sample weights
sample_weights = get_sample_weights(train)

# Create DataLoader with weighted sampling
from torch.utils.data import DataLoader, WeightedRandomSampler

train_dataset_weighted = MultiModalityDataset(train, get_transforms(IMAGE_SIZE, train=True))

weighted_sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

train_loader_weighted = DataLoader(
    train_dataset_weighted,
    batch_size=BATCH_SIZE,
    sampler=weighted_sampler,  # Use sampler instead of shuffle
    num_workers=2
)

print(f"✅ Weighted sampler created!")
print(f"   Rare classes will be sampled more frequently during training.")

## ⚖️ Strategy 3: Weighted Loss Function

Calculate class weights to use in CrossEntropyLoss for penalizing misclassification of rare classes more.

In [ ]:
# Calculate class weights for loss function
class_weights = calculate_class_weights(train, num_classes=len(class_names))

print(f"Class weights shape: {class_weights.shape}")
print(f"Sample weights (first 5 classes): {class_weights[:5]}")

# Use in training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion_weighted = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))

print(f"\n✅ Use criterion_weighted in your training loop!")

## 📊 Summary: Which Strategy to Use?

| Strategy | Pros | Cons | Best For |
|----------|------|------|----------|
| **No balancing** | Simple, uses all data | May bias toward majority classes | Balanced datasets |
| **Uniform sampling** | Perfect balance, fast | Discards data from larger classes | Severe imbalance, plenty of data |
| **Weighted sampler** | Uses all data, no loss modification | May oversample minority too much | Training data only, moderate imbalance |
| **Weighted loss** | Uses all data, theoretically sound | Harder to tune | Works well with any imbalance level |

**Recommendation**: Start with **weighted loss** as it's most flexible. Try **weighted sampler** if that doesn't help.